In [1]:
import pandas as pd
from argparse import Namespace
from sklearn.metrics import classification_report
from transformers import pipeline

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [7]:
df = pd.read_csv('final_sent_score.csv',encoding='cp949')
df.dropna(inplace=True)
df

,date,title,content,sent_score
0,2024-05-20,코스피 상장사 현금배당 10년 만에 15조→41조 | 한국경제,유가증권시장 상장사의 현금배당 규모가 최근 10년간 세 배 가까이 늘어난 것으로 나...,27
1,2024-05-20,"코스피 편입 종목, 1분기 연결 영업이익 84%↑…코스닥은 4% 줄어 | 한국경제",사진=한경DB 사진=한경DB 유가증권시장 상장사들은 지난 1분기 실적이 1년 전보...,-7
2,2024-05-21,"코스피, 기관·외국인 매도세에 2720선 후퇴…대형주 약세 | 한국경제",17일 서울 중구 하나은행 본점 딜링룸./사진=뉴스1 17일 서울 중구 하나은행 본...,6
3,2024-05-20,"생큐 반도체…코스피 상장사 영업이익, 1년 만에 84％ 점프 | 한국경제",사진=게티이미지뱅크 사진=게티이미지뱅크 유가증권시장 상장사 영업이익이 전년 대비 ...,6
4,2024-05-21,"코스피, 소폭 하락하며 2730선 횡보…화장품株 들썩 | 한국경제",21일 오전 서울 중구 하나은행 딜링룸./사진=연합뉴스 21일 오전 서울 중구 하나...,-4
...,...,...,...,...
7579,2024-05-29,요즘 짜장면도 8000원인데…2만원 돼도 사 먹을래요 | 한국경제,서울 중구에 위치한 평양냉면 전문점 우래옥에 길게 늘어선 줄 / 사진=성진우 기자 ...,4
7580,2016-07-08,"코스피, 오후에도 1960선 약세 지속…삼성전자 1%대↑ | 한국경제",코스피지수가 오후 들어서도 1960선에서 약세를 지속하고 있다. 코스닥지수도 하락세...,-6
7581,2024-05-30,기사님들이 엄지척 입소문 탔다…인기 폭발 전기차 택시 | 한국경제,더 뉴 아이오닉5 더 뉴 아이오닉5 현대차그룹의 전기차 전용 플랫폼인 E-GMP를...,10
7582,2016-07-08,"코스피, 기관·외인 팔자에 1950선 낙폭 확대…코스닥 690선 내줘 | 한국경제",코스피지수가 1950선에서 약세를 나타내고 있다. 코스닥지수도 장중 하락세로 돌아섰...,-3


In [8]:
from tqdm import tqdm

sent_score_ls = []
pst_ls = []
neg_ls = []
# tqdm을 사용하여 진행 상황 표시
for i in tqdm(len(df), desc="Processing articles"):
    try:
        content = df['content'][i].split('. ')
        content = [e for e in content if len(e) <= 512]
        sent_ls = []
        sentiment = classifier(content)
        
        for j in range(len(sentiment)):
            sent_ls.append(sentiment[j]['label'])
        
        title_sentiment = classifier(df['title'][i])
        sent_ls.append(title_sentiment[0]['label'])
        
        positive = sent_ls.count('positive')
        negative = sent_ls.count('negative')
        pst_ls.append(positive)
        neg_ls.append(negative)
        sent_score = positive - negative
        sent_score_ls.append(sent_score)
    except KeyError:
        pass

Processing articles:   0%|          | 0/3 [00:00<?, ?it/s]

Processing articles: 100%|██████████| 3/3 [00:09<00:00,  3.19s/it]


[30, 9, 13]

In [ ]:
df['positive'] = pst_ls
df['negative'] = neg_ls
df